In [2]:
# !pip install torch diffusers transformers accelerate Pillow tqdm
# import torch
# from transformers import CLIPTextModel, CLIPTokenizer
# from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
# from PIL import Image
# from tqdm.auto import tqdm

# # Initialize models globally
# def initialize_models():
#     global vae, tokenizer, text_encoder, unet, scheduler, torch_device
#     torch_device = "cuda" if torch.cuda.is_available() else "cpu"

#     vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(torch_device)
#     tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
#     text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(torch_device)
#     unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet").to(torch_device)
#     scheduler = LMSDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

# # Call this once to initialize the models
# initialize_models()

# def generate_image_from_prompt(prompt: str, seed: int = 32, height: int = 512, width: int = 512, num_inference_steps: int = 100, guidance_scale: float = 7.5) -> Image:
#     global vae, tokenizer, text_encoder, unet, scheduler, torch_device

#     # Prepare text embeddings
#     text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
#     with torch.no_grad():
#         text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
#     max_length = text_input.input_ids.shape[-1]
#     uncond_input = tokenizer([""] * 1, padding="max_length", max_length=max_length, return_tensors="pt")
#     with torch.no_grad():
#         uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
#     text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

#     # Initialize latents
#     generator = torch.manual_seed(seed)
#     latents = torch.randn(
#         (1, unet.in_channels, height // 8, width // 8),
#         generator=generator,
#     ).to(torch_device)
#     latents = latents * scheduler.init_noise_sigma
#     scheduler.set_timesteps(num_inference_steps)

#     # Denoising loop
#     for t in tqdm(scheduler.timesteps):
#         latent_model_input = torch.cat([latents] * 2)
#         latent_model_input = scheduler.scale_model_input(latent_model_input, t)
#         with torch.no_grad():
#             noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
#         noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
#         noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
#         latents = scheduler.step(noise_pred, t, latents).prev_sample

#     # Decode the image
#     latents = 1 / 0.18215 * latents
#     with torch.no_grad():
#         image = vae.decode(latents).sample
#     image = torch.tensor(image)
#     image = (image / 2 + 0.5).clamp(0, 1)
#     image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
#     images = (image * 255).round().astype("uint8")
#     pil_image = Image.fromarray(images[0])

#     return pil_image

# # Example usage
# prompt = "Create a highly detailed and realistic image showing the leg of a person with brown skin. The leg should display a typical ringworm infection, characterized by a clearly visible rash."
# generated_image = generate_image_from_prompt(prompt)
# generated_image


In [3]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.4 MB/s eta 0:00:00


In [4]:
import re
import torch, logging

import torch
from PIL import Image
import io

## disable warnings
logging.disable(logging.WARNING)

## Imaging  library
from PIL import Image
from torchvision import transforms as tfms

## Basic libraries
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython.display import display
import shutil
import os

## For video display
from IPython.display import HTML
from base64 import b64encode


## Import the CLIP artifacts
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from IPython.display import display, clear_output
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def magic_prompt(color, rash_type, body_part):
    # Construct the detailed prompt using the extracted values
    return f"Create a highly detailed and realistic image showing the {body_part} of a person with {color} skin. The {body_part} should display a typical {rash_type} infection, characterized by a clearly visible rash."


## Helper functions
def load_image(p):
    '''
    Function to load images from a defined path
    '''
    return Image.open(p).convert('RGB').resize((512,512))

def pil_to_latents(image):
    '''
    Function to convert image to latents
    '''
    init_image = tfms.ToTensor()(image).unsqueeze(0) * 2.0 - 1.0
    init_image = init_image.to(device="cuda", dtype=torch.float16)
    init_latent_dist = vae.encode(init_image).latent_dist.sample() * 0.18215
    return init_latent_dist

def latents_to_pil(latents):
    '''
    Function to convert latents to images
    '''
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

In [6]:
def text_enc(prompts, maxlen=None):
    '''
    A function to take a texual promt and convert it into embeddings
    '''
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt")
    return text_encoder(inp.input_ids.to("cuda"))[0].half()


In [7]:
def prompt_2_img(prompts, g=7.5, seed=100, steps=70, dim=512, save_int=True):
    """
    Diffusion process to convert prompt to image, modified to yield images for Streamlit.
    """

    bs = len(prompts)
    text = text_enc(prompts)
    uncond = text_enc([""] * bs, text.shape[1])
    emb = torch.cat([uncond, text])

    if seed:
        torch.manual_seed(seed)

    latents = torch.randn((bs, unet.in_channels, dim//8, dim//8))
    scheduler.set_timesteps(steps)
    latents = latents.to("cuda").half() * scheduler.init_noise_sigma

    for i, ts in enumerate(scheduler.timesteps):
        inp = scheduler.scale_model_input(torch.cat([latents] * 2), ts)
        with torch.no_grad():
            u, t = unet(inp, ts, encoder_hidden_states=emb).sample.chunk(2)
        pred = u + g*(t-u)
        latents = scheduler.step(pred, ts, latents).prev_sample

        if save_int and i % (steps // 70) == 0:  # Yield 10 images throughout the process
            image = latents_to_pil(latents)[0]
            buf = io.BytesIO()
            image.save(buf, format="JPEG")
            byte_im = buf.getvalue()
            yield byte_im  # Yield image in bytes format for Streamlit to display

    final_image = latents_to_pil(latents)
    final_buf = io.BytesIO()
    final_image[0].save(final_buf, format="JPEG")
    final_byte_im = final_buf.getvalue()
    yield final_byte_im  # Yield the final image

In [8]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

## Initiating the VAE
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", torch_dtype=torch.float16).to("cuda")

## Initializing a scheduler and Setting number of sampling steps
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
scheduler.set_timesteps(50)

## Initializing the U-Net model
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [9]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask_cors

In [14]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN_HERE' with your actual authtoken
ngrok.set_auth_token("2k7TLBxJagqxkXOByAQHCMlUVHG_59eFBXAnCvT5fBCHmi8Ez")

# Open a tunnel on port 5000
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://localhost:5000\"")


 * ngrok tunnel "NgrokTunnel: "https://5d54-34-124-236-71.ngrok-free.app" -> "http://localhost:5000"" -> "http://localhost:5000"


In [16]:
!pip install flask flask-cors pyngrok


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from PIL import Image
import base64
import io

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    skin_rash_type = data.get('skinRashType')
    skin_color = data.get('skinColor')
    body_part = data.get('bodyPart')
    detailed_prompt = magic_prompt(skin_color, skin_rash_type, body_part)
    generator_new = prompt_2_img([detailed_prompt], save_int=True)
    total_steps = 70
    try:
      for i, image_bytes in enumerate(generator_new):
        print(f"Processing image {i}")
        pg = min((step) / total_steps, 100)
        if step%13 == 0:
            clear_output(wait=True)
            print(f"Progress: {pg:.2f}%")
    except Exception as e:
        print(f"An error occurred: {e}")
    for step, image_bytes in enumerate(generator_new):
            pg = min((step) / total_steps, 100)
            if step%13 == 0:
                clear_output(wait=True)
                print(f"Progress: {pg:.2f}%")
    # try:
    #   image_bytes = next(generator_new)
    # except StopIteration:
    #   print("No more images.")
    # except Exception as e:
    #     print(f"An error occurred: {e}")
    img_str = base64.b64encode(image_bytes).decode("utf-8")

    return jsonify({"image": img_str})


def start_flask_app():
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://localhost:5000\"")

    app.run(port=5000)

if __name__ == "__main__":
    start_flask_app()

Progress: 0.93%
